**[Intro to Game AI and Reinforcement Learning Home Page](https://www.kaggle.com/learn/intro-to-game-ai-and-reinforcement-learning)**

---


# Setting up training buddies

In [ ]:
%load_ext Cython

In [ ]:
%%cython
cimport cython
from libc.stdlib cimport malloc
from libc.stdlib cimport rand, RAND_MAX

# Checks for 3 and 4 in a row
cdef int get_score(int[42] grid, int mark, int[42] prev_grid, int depth):

    cdef int t, row, col, summe
    cdef int num_threes = 0
    cdef int num_threes_opp = 0
     
    # horizontal
    for row in range(6):
        col = 0
        while col < 4:
            summe = 0
            for t in range(4):
                summe += grid[row * 7 + col + t]
    
            if summe < 3 and summe >= 0:
                col += 3 - summe
                continue
                
            if summe > -3 and summe < 0:
                col += 3 + summe
                continue
                
            col += 1    
            summe *= mark
            if summe == 3:
                num_threes += 1
                continue
            if summe == -3:
                num_threes_opp += 1
    
    # vertical
    for col in range(7):
        for row in range(3):
            summe = 0
            for t in range(4):
                summe += grid[(row+t) * 7 + col]
            
            if summe == 0:
                break
                
            summe *= mark
            if summe == 3:
                num_threes += 1
                continue
            if summe == -3:
                num_threes_opp += 1

    # positive diagonal
    for row in range(3):
        col = 0
        while col < 4:
            summe = 0
            for t in range(4):
                summe += grid[(row+t) * 7 + col + t]
    
            if summe < 3 and summe >= 0:
                col += 3 - summe
                continue
                
            if summe > -3 and summe < 0:
                col += 3 + summe
                continue
                
            col += 1   
            summe *= mark
            if summe == 3:
                num_threes += 1
                continue
            if summe == -3:
                num_threes_opp += 1

    # negative diagonal
    for row in range(3,6):
        col = 0
        while col < 4:
            summe = 0
            for t in range(4):
                summe += grid[(row-t) * 7 + col + t]
    
            if summe < 3 and summe >= 0:
                col += 3 - summe
                continue
                
            if summe > -3 and summe < 0:
                col += 3 + summe
                continue
                
            col += 1   
            summe *= mark
            if summe == 3:
                num_threes += 1
                continue
            if summe == -3:
                num_threes_opp += 1
                  
    return num_threes - 2 * num_threes_opp # Alternatively weigh opponents higher or lower


# Checks if it is a terminal position, if true it returns the score
cdef int is_terminal_node(int[42] board, int column, int mark, int row, int player_mark, int depth):
    
    cdef int i = 0
    cdef int j = 0
    cdef int col = 0
    
    # To check if board is full
    for col in range(7):
        if board[col] == 0:
            break
        col += 1
    
    # vertical
    if row < 3:
        for i in range(1, 4):
            if board[column + (row+i) * 7] != mark:
                break
            i += 1
    if i == 4:
        if player_mark == mark:
            return 1000 + depth # depth added, so that it chooses the faster option to win
        else:
            return -1000 - depth
    
    # horizontal
    for i in range(1, 4):
        if (column + i) >= 7 or board[column + i + (row) * 7] != mark:
            break
        i += 1
    for j in range(1, 4):
        if (column - j) < 0 or board[column - j + (row) * 7] != mark:
            break
        j += 1
    if (i + j) >= 5:
        if player_mark == mark:
            return 1000 + depth
        else:
            return -1000 - depth
    
    # top left diagonal
    for i in range(1, 4):
        if (column + i) >= 7 or (row + i) >= 6 or board[column + i + (row + i) * 7] != mark:
            break
        i += 1
    for j in range(1, 4):
        if (column - j) < 0 or(row - j) < 0 or board[column - j + (row - j) * 7] != mark:
            break
        j += 1
    if (i + j) >= 5:
        if player_mark == mark:
            return 1000 + depth
        else:
            return -1000 - depth
    
    # top right diagonal
    for i in range(1, 4):
        if (column + i) >= 7 or (row - i) < 0 or board[column + i + (row - i) * 7] != mark:
            break
        i += 1
    for j in range(1, 4):
        if (column - j) < 0 or(row + j) >= 6 or board[column - j + (row + j) * 7] != mark:
            break
        j += 1
    if (i + j) >= 5:
        if player_mark == mark:
            return 1000 + depth
        else:
            return -1000 - depth
    
    if col == 7:
        return 1 # draw
    return 0 # nobody has won so far


# Initial move is scored with minimax
cdef int score_move(int[42] grid, int col, int mark, int nsteps):

    cdef int[42] next_grid = grid
    cdef int row, row2, column
    cdef int[42] child
    
    for row in range(5, -1, -1):
        if next_grid[7 * row + col] == 0:
            next_grid[7 * row + col] = mark # drop mark
            break
    
    if nsteps > 2: # check if there is an obvious move
        is_terminal = is_terminal_node(next_grid, col, mark, row, mark, nsteps-1)
        if is_terminal != 0:
            return is_terminal

        for column in range(7):
            if next_grid[column] != 0:
                continue
            child = next_grid
            for row2 in range(5, -1, -1):
                if child[7 * row2 + column] == 0:
                    child[7 * row2 + column] = mark*(-1)
                    break

            is_terminal = is_terminal_node(child, column, mark*(-1), row2, mark, nsteps-2)
            if is_terminal != 0:
                return is_terminal + (col == column) #added in case the opponent makes a mistake
        
    cdef int alpha = - 10000000
    cdef int beta = 10000000
    return minimax(next_grid, nsteps-1, 0, mark, grid, alpha, beta, col, row)


# Minimax agent with alpha-beta pruning
cdef int minimax(int[42] node, int depth, int maximizingPlayer, int mark, int[42] grid, int alpha, int beta, int column, int newrow):
    
    cdef int is_terminal 
    if maximizingPlayer:
        is_terminal = is_terminal_node(node, column, mark*(-1), newrow, mark, depth)
        if is_terminal != 0:
            return is_terminal
    if maximizingPlayer == 0:
        is_terminal = is_terminal_node(node, column, mark, newrow, mark, depth)
        if is_terminal != 0:
            return is_terminal

    cdef int value, col, row
    cdef int[42] child
    
    if depth == 0:
        return get_score(node, mark, grid, depth)

    if maximizingPlayer:
        value = -1000000
        for col in range(7):
            if node[col] != 0:
                continue
            child = node
            for row in range(5, -1, -1):
                if child[7 * row + col] == 0:
                    child[7 * row + col] = mark 
                    break
            value = max(value, minimax(child, depth-1, 0, mark, grid, alpha, beta, col, row))
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return value
    else:
        value = 1000000
        for col in range(7):
            if node[col] != 0:
                continue
            child = node
            for row in range(5, -1, -1):
                if child[7 * row + col] == 0:
                    child[7 * row + col] = mark*(-1)
                    break
            value = min(value, minimax(child, depth-1, 1, mark, grid, alpha, beta, col, row))
            beta = min(beta, value)
            if beta <= alpha:
                break
        return value
    

# define the agent   
@cython.cdivision(True)
cpdef int agen(list grid, int mark, int N_STEPS):
    
    if mark == 2:
        mark = -1
        
    cdef int num_max = 1
    cdef int col, sc, i
    cdef int maxsc = -1000001
    cdef int[7] score = [-10000, -10000, -10000, -10000, -10000, -10000, -10000]

    cdef int *c_grid
    
    c_grid = <int *>malloc(42*cython.sizeof(int))
    for i in range(42):
        if grid[i] == 2:
            c_grid[i] = -1
            continue
        c_grid[i] = grid[i]
    
    for col in range(7):
        if c_grid[col] == 0:
            sc = score_move(c_grid, col, mark, N_STEPS)
            if sc == maxsc:
                num_max += 1
                
            if sc > maxsc:
                maxsc = sc
                num_max = 1
                
            score[col] = sc
            
    cdef int choice = int(rand()/(RAND_MAX/num_max))
    cdef int indx = 0
    
    #print(score, mark)

    for i in range(7):
        if score[i] == maxsc:
            if choice == indx:
                return i  
            indx += 1
     
    return 0 # shouldn't be necessary

In [ ]:
from agents import matrix_agent

def agentc1(obs, conf):
    return agen(obs.board, obs.mark, 1)

def agentc2(obs, conf):
    return agen(obs.board, obs.mark, 2)

def agentc3(obs, conf):
    return agen(obs.board, obs.mark, 3)

def agentc5(obs, conf):
    return agen(obs.board, obs.mark, 5)

def agentc7(obs, conf):
    return agen(obs.board, obs.mark, 7)

def agentc9(obs, conf):
    return agen(obs.board, obs.mark, 9)

def agentc11(obs, conf):
    return agen(obs.board, obs.mark, 11)

# Creating the gym environment 

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from kaggle_environments import make, evaluate
from gym import spaces

import tensorflow as tf


from stable_baselines.bench import Monitor 
from stable_baselines.common.vec_env import DummyVecEnv

from stable_baselines import PPO2 
from stable_baselines.common.policies import CnnPolicy
from stable_baselines.common.callbacks import EvalCallback

from numpy.random import choice


# We are always playing mark 1
def board_flip(mark, board):
    if mark == 1:
        return board

    for i in range(board.shape[0]):
        for j in range(board.shape[1]):
            if board[i, j, 0] != 0:
                board[i, j, 0] = board[i, j, 0]%2 + 1

    return board

class ConnectFourGym:
    def __init__(self, opponent_pool=np.asarray(['random']), distribution='even'):
        self.ks_env = make("connectx", debug=True)
        self.rows = self.ks_env.configuration.rows
        self.columns = self.ks_env.configuration.columns
        self.inarow = self.ks_env.configuration.inarow
        # Learn about spaces here: http://gym.openai.com/docs/#spaces
        self.action_space = spaces.Discrete(self.columns)
        self.observation_space = spaces.Box(low=0, high=2, 
                                            shape=(self.rows,self.columns,1), dtype=np.int)
        # Tuple corresponding to the min and max possible rewards
        self.reward_range = (-10, 1)
        # StableBaselines throws error if these are not defined
        self.spec = None
        self.metadata = None
        self.last_action = -1
        self.iter = 0
        self.opponent_pool = opponent_pool
        self.distribution = distribution
        self.init_env()

    
    def init_env(self):
        if self.distribution == 'even':
            distribution = [1.0 / len(self.opponent_pool)] * len(self.opponent_pool)
        else:
            distribution = self.distribution
            
        opponent = choice(self.opponent_pool, 1, p=distribution)[0]
        
        if self.iter % 2:
            self.env = self.ks_env.train([None, opponent])
        else:
            self.env = self.ks_env.train([opponent, None])
        
    
    def reset(self):
        self.iter += 1
        self.init_env()
        self.obs = self.env.reset()
        self.last_action = -1
        return board_flip(self.obs.mark, np.array(self.obs['board']).reshape(self.rows,self.columns,1))
    
    def check_window(self, window, num_discs, piece):
        return (window.count(piece) == num_discs)# and window.count(0) == self.inarow-num_discs)
    
    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(self, grid, num_discs, piece):
        num_windows = 0
        # horizontal
        for row in range(self.rows):
            for col in range(self.columns-(self.inarow-1)):
                window = list(grid[row, col:col+self.inarow])
                if self.check_window(window, num_discs, piece):
                    num_windows += 1
        # vertical
        for row in range(self.rows-(self.inarow-1)):
            for col in range(self.columns):
                window = list(grid[row:row+self.inarow, col])
                if self.check_window(window, num_discs, piece):
                    num_windows += 1
        # positive diagonal
        for row in range(self.rows-(self.inarow-1)):
            for col in range(self.columns-(self.inarow-1)):
                window = list(grid[range(row, row+self.inarow), range(col, col+self.inarow)])
                if self.check_window(window, num_discs, piece):
                    num_windows += 1
        # negative diagonal
        for row in range(self.inarow-1, self.rows):
            for col in range(self.columns-(self.inarow-1)):
                window = list(grid[range(row, row-self.inarow, -1), range(col, col+self.inarow)])
                if self.check_window(window, num_discs, piece):
                    num_windows += 1
                    
        return num_windows
    
    def calculate_heuristic_reward(self):
        grid = np.asarray(self.obs.board).reshape(self.rows, self.columns)
        twos = self.count_windows(grid, 2, self.obs.mark)
        threes = self.count_windows(grid, 3, self.obs.mark)
        
        reward = (twos - self.last_twos) * 1 + (threes - self.last_threes) * 3
        if reward < 0:
            raise AssertionError(f'\nmark:   {self.obs.mark}   last_action: {self.last_action}\n' + 
                                 f'twos:   {twos}   last_twos: {self.last_twos}\n' + 
                                 f'threes: {threes} last_threes: {self.last_threes}\n' +
                                 f'Board:  \n{np.array_str(grid)}')
            
        self.last_twos = twos
        self.last_threes = threes
        
        return reward
    
    def change_reward(self, old_reward, done):
        if old_reward == 1: # The agent won the game
            return 1
        elif done: # The opponent won the game
            return -1
        else: # Reward 1/42
            return 1/(self.rows*self.columns)
    
    def step(self, action):
        # Check if agent's move is valid
        is_valid = (self.obs['board'][int(action)] == 0)
        if is_valid: # Play the move
            self.obs, old_reward, done, _ = self.env.step(int(action))
            reward = self.change_reward(old_reward, done)
        else: # End the game and penalize agent
            reward, done, _ = -10, True, {}
            
        new_board = board_flip(self.obs.mark, np.array(self.obs['board']).reshape(self.rows,self.columns,1)) 
        return new_board, reward, done, _
    

    
"""    
    def change_reward(self, old_reward, done):
        if old_reward == 1: # The agent won the game
            return 100
        elif done: # The opponent won the game
            return -100
        else: # Reward 10/42
            still_alive_reward = 10 / (self.rows*self.columns)
            #move_reward = self.calculate_heuristic_reward()
            return still_alive_reward# + move_reward
    
    def step(self, action):
        # Check if agent's move is valid
        self.last_action = action
        is_valid = (self.obs['board'][int(action)] == 0)
        if is_valid: # Play the move
            self.obs, old_reward, done, _ = self.env.step(int(action))
            reward = self.change_reward(old_reward, done)
        else: # End the game and penalize agent
            reward, done, _ = -1000, True, {}
        return np.array(self.obs['board']).reshape(self.rows,self.columns,1), reward, done, _
"""    
    
    
# Create ConnectFour environment
env = ConnectFourGym([matrix_agent, 'random', agentc1, agentc2, agentc3, agentc5])

# Create directory for logging training information
log_dir = "logtf1/"
os.makedirs(log_dir, exist_ok=True)

# Logging progress
monitor_env = Monitor(env, log_dir, allow_early_resets=True)

# Create a vectorized environment
vec_env = DummyVecEnv([lambda: monitor_env])



# Model definition

In [ ]:
from stable_baselines.a2c.utils import conv, linear, conv_to_fc
from tensorflow.layers import Dropout, BatchNormalization, Dense, Conv2D
import tensorflow as tf


"""
args = dotdict({
    'lr': 0.001,
    'dropout': 0.3,
    'epochs': 5,
    'batch_size': 64,
    'num_channels': 64,
})
"""


NUM_CHANNELS = 64

BN1 = BatchNormalization()
BN2 = BatchNormalization()
BN3 = BatchNormalization()
BN4 = BatchNormalization()
BN5 = BatchNormalization()
BN6 = BatchNormalization()


CONV1 = Conv2D(NUM_CHANNELS, kernel_size=3, strides=1, padding='same')
CONV2 = Conv2D(NUM_CHANNELS, kernel_size=3, strides=1, padding='same')
CONV3 = Conv2D(NUM_CHANNELS, kernel_size=3, strides=1)
CONV4 = Conv2D(NUM_CHANNELS, kernel_size=3, strides=1)

FC1 = Dense(128)
FC2 = Dense(64)
FC3 = Dense(7)

DROP1 = Dropout(0.3)
DROP2 = Dropout(0.3)


# 6x7 input
# https://github.com/PaddlePaddle/PARL/blob/0915559a1dd1b9de74ddd2b261e2a4accd0cd96a/benchmark/torch/AlphaZero/submission_template.py#L496
def modified_cnn(inputs, **kwargs):
    relu = tf.nn.relu
    log_softmax = tf.nn.log_softmax
    
    
    layer_1_out = relu(BN1(CONV1(inputs)))
    layer_2_out = relu(BN2(CONV2(layer_1_out)))
    layer_3_out = relu(BN3(CONV3(layer_2_out)))
    layer_4_out = relu(BN4(CONV4(layer_3_out)))
    
    # 3 is width - 4 due to convolition filters, 2 is same for height
    flattened = tf.reshape(layer_4_out, [-1, NUM_CHANNELS * 3 * 2]) 
    
    layer_5_out = DROP1(relu(BN5(FC1(flattened))))
    layer_6_out = DROP2(relu(BN6(FC2(layer_5_out))))
    
    return log_softmax(FC3(layer_6_out))  

# https://www.kaggle.com/c/connectx/discussion/128591
class CustomCnnPolicy(CnnPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomCnnPolicy, self).__init__(*args, **kwargs, cnn_extractor=modified_cnn)

In [ ]:
from stable_baselines.common.callbacks import BaseCallback
from common import get_win_percentages_and_score, serializeAndCompress

class SaveBestModelCallback(BaseCallback):
    """
    :param verbose: (int) Verbosity level 0: not output 1: info 2: debug
    """
    def __init__(self, model_basename, save_frequency, test_agents, verbose=0):
        super(SaveBestModelCallback, self).__init__(verbose)
        # Those variables will be accessible in the callback
        # (they are defined in the base class)
        # The RL model
        # self.model = None  # type: BaseRLModel
        # An alias for self.model.get_env(), the environment used for training
        # self.training_env = None  # type: Union[gym.Env, VecEnv, None]
        # Number of time the callback was called
        # self.n_calls = 0  # type: int
        # self.num_timesteps = 0  # type: int
        # local and global variables
        # self.locals = None  # type: Dict[str, Any]
        # self.globals = None  # type: Dict[str, Any]
        # The logger object, used to report things in the terminal
        # self.logger = None  # type: logger.Logger
        # # Sometimes, for event callback, it is useful
        # # to have access to the parent object
        # self.parent = None  # type: Optional[BaseCallback]
        self.step_counter = 0
        self.best_value = -np.inf
        self.model_basename = model_basename
        self.save_frequency = save_frequency
        self.test_agents = test_agents
        

    def _on_training_start(self) -> None:
        """
        This method is called before the first rollout starts.
        """
        pass

    def _on_rollout_start(self) -> None:
        """
        A rollout is the collection of environment interaction
        using the current policy.
        This event is triggered before collecting new samples.
        """
        pass

    def _on_step(self) -> bool:
        """
        This method will be called by the model after each call to `env.step()`.

        For child callback (of an `EventCallback`), this will be called
        when the event is triggered.

        :return: (bool) If the callback returns False, training is aborted early.
        
        """
        self.step_counter += 1
        if self.step_counter % self.save_frequency == 0:
            print(f'Running check model check on step {self.step_counter}')
            def trained_agent(obs, config):
                # Use the best model to select a column
                grid = board_flip(obs.mark, np.array(obs['board']).reshape(6,7,1))
                col, _ = self.model.predict(grid, deterministic=True)
                # Check if selected column is valid
                is_valid = (obs['board'][int(col)] == 0)
                # If not valid, select random move. 
                if is_valid:
                    return int(col)
                else:
                    return random.choice([col for col in range(config.columns) if obs.board[int(col)] == 0])
            
            
            score = sum([get_win_percentages_and_score(trained_agent, test_agent, silent=True) for test_agent in self.test_agents])
            if score > self.best_value:
                self.best_value = score
                print('=' * 80)
                print(f'New best agent found with score {score}! Agent encoded:')
                model_serialized = serializeAndCompress(self.model.get_parameters())
                print(model_serialized)
                with open(self.model_basename + str(self.step_counter) + '.model', 'wb') as f:
                    f.write(model_serialized)
                
        
        return True

    def _on_rollout_end(self) -> None:
        """
        This event is triggered before updating the policy.
        """
        pass

    def _on_training_end(self) -> None:
        """
        This event is triggered before exiting the `learn()` method.
        """
        pass

Next, run the code cell below to train an agent with PPO and view how the rewards evolved during training.  This code is identical to the code from the tutorial.

In [ ]:
# Initialize agent
# Try CnnPolicy and MlpPolicy
# https://www.kaggle.com/toshikazuwatanabe/connect4-make-submission-with-stable-baselines3/comments


eval_callback = SaveBestModelCallback('RDaneelConnect4_', 1000, ['random', agentc1, agentc3, agentc5, matrix_agent])

model = PPO2(CustomCnnPolicy, vec_env, verbose=1)


# Train agent
model.learn(total_timesteps=30000000, callback=eval_callback)

#vec_env.close()

In [ ]:
# Plot cumulative reward

plt.figure(figsize = (20,20))


with open(os.path.join(log_dir, "monitor.csv"), 'rt') as fh:    
    firstline = fh.readline()
    assert firstline[0] == '#'
    df = pd.read_csv(fh, index_col=None)['r']
df.rolling(window=100).mean().plot()
plt.show()

In [ ]:
from time import sleep

def dqn_agent(obs, config):
    # Use the best model to select a column
    grid = board_flip(obs.mark, np.array(obs['board']).reshape(6,7,1))
    col, _ = model.predict(grid, deterministic=True)
    # Check if selected column is valid
    is_valid = (obs['board'][int(col)] == 0)
    # If not valid, select random move. 
    if is_valid:
        return int(col)
    else:
        grid = grid.reshape(6, 7)
        #sleep(2)
        #print(f'Illegal move attempted! Move: {col}, Boardf:\n{grid}')
        return random.choice([col for col in range(config.columns) if obs.board[int(col)] == 0])
    

env = make("connectx", debug=True)
env.play([dqn_agent, None], width=500, height=450)

In [ ]:
get_win_percentages_and_score('random', dqn_agent)
get_win_percentages_and_score(agentc1, dqn_agent)
get_win_percentages_and_score(agentc2, dqn_agent)
get_win_percentages_and_score(agentc3, dqn_agent)
get_win_percentages_and_score(agentc5, dqn_agent)
get_win_percentages_and_score(agentc7, dqn_agent)
get_win_percentages_and_score(matrix_agent, dqn_agent)

In [ ]:
compressed = serializeAndCompress(model.get_parameters())
print(compressed)

If your agent trained well, the plot (which shows average cumulative rewards) should increase over time.

Once you have verified that the code runs, try making amendments to see if you can get increased performance.  You might like to:
- change `PPO1` to `A2C` (or `ACER` or `ACKTR` or `TRPO`) when defining the model in this line of code: `model = PPO1(CustomCnnPolicy, vec_env, verbose=0)`.  This will let you see how performance can be affected by changing the algorithm from Proximal Policy Optimization [PPO] to one of:
  - Advantage Actor-Critic (A2C),
  - or Actor-Critic with Experience Replay (ACER),
  - Actor Critic using Kronecker-factored Trust Region (ACKTR), or 
  - Trust Region Policy Optimization (TRPO).
- modify the `change_reward()` method in the `ConnectFourGym` class to change the rewards that the agent receives in different conditions.  You may also need to modify `self.reward_range` in the `__init__` method (this tuple should always correspond to the minimum and maximum reward that the agent can receive).
- change `agent2` to a different agent when creating the ConnectFour environment with `env = ConnectFourGym(agent2="random")`.  For instance, you might like to use the `"negamax"` agent, or a different, custom agent.  Note that the smarter you make the opponent, the harder it will be for your agent to train!

# Congratulations!

You have completed the course, and it's time to put your new skills to work!  

The next step is to apply what you've learned to a **[more complex game: Halite](https://www.kaggle.com/c/halite)**.  For a step-by-step tutorial in how to make your first submission to this competition, **[check out the bonus lesson](https://www.kaggle.com/alexisbcook/getting-started-with-halite)**!

You can find more games as they're released on the **[Kaggle Simulations page](https://www.kaggle.com/simulations)**.

As we did in the course, we recommend that you start simple, with an agent that follows your precise instructions.  This will allow you to learn more about the mechanics of the game and to build intuition for what makes a good agent.  Then, gradually increase the complexity of your agents to climb the leaderboard!

---
**[Intro to Game AI and Reinforcement Learning Home Page](https://www.kaggle.com/learn/intro-to-game-ai-and-reinforcement-learning)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*